In [ ]:
import altair as alt
alt.data_transformers.enable("vegafusion",message = False)
alt.renderers.enable('mimetype',message = False)
alt.renderers.enable('jupyterlab',message = False)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Abstract

Our project aims to predict the probability of a wicket on a delivery in Cricket. We will look at ball-by-ball factors that could potentially affect this probability and see how the wicket probability changes with game conditions. Forecasting methods for cricket use complex regression models as discussed by @asif2016play, which is why we decided against predicting win probability, a common prediction model used for sports.

## Introduction

Cricket is a bat and ball sport involving two teams of eleven players each. Originating in the 16th century, Cricket today is commonly played in three formats: Tests, One Day Internationals and T20 Internationals. Our analysis will be specifically focusing on T20 International games and looking at the conditions of those games to analyze our prediction probability. In a T20 game, one team will bat first and set a target number of runs in 20 overs or until 10 wickets are lost. The other team will bat second, and to win the game, they must exceed the target set by the second team in 20 overs or less. Each of the 11 players in the team will need to bat as more wickets are lost, but some bat more often than others, given all wickets are not always lost. Only some players are bowlers. There are multiple different ways of getting a wicket, and these can be affected by the following factors:

1. Bowler: The skill of a bowler can affect the probability
2. Batter: Similarly, the skill of a batter can affect the probability
3. Inning: Determines which team is batting and bowling
4. Over: An over consists of 6 deliveries thrown by the bowler from one end of the pitch to the batter on the other end. The over can determine how much risk a batter is willing to take with his next ball.
5. Ball in the over: The specific delivery in the over. Batters are more likely to take risks towards the end of an over compared to the beginning.
6. Runs scored till now: Looks at the amount of runs scored in the inning up until the current ball. Fewer runs scored may mean the batter is more likely to take risks in order to get more.
7. Powerplay: A powerplay is when there are fielding restrictions placed on the bowling team, making it harder for them to get a wicket. 

We will be looking at the above factors along with some others to judge what factors may be best for our model in the Exploratory Data Analysis section. We will be reading our data in json files from the [Cricsheet](https://cricsheet.org/downloads/) website, and converting it to our desired `csv` format using the scripts in the `cricsheet_json_parsing.ipynb` file which can be found in the `src` folder.

## Methods

Firstly, we will perform some exploratory data analysis on the training data to get a better understanding of the dataset and what relationships there are with our target variable.

### Exploratory Data Analysis

The dataset contains 28 columns and 510652 observations related to cricket. The ratio of quantitative to categorical (ordinal and nominal) data is approximately 50:50. Note that the columns **player_out, player_out_id, fielder_name, fielders_id, and wicket_type** show a significant amount of missing values. Player_out and player_out_id may be missing due to inapplicability (e.g., no one is out), which results in fielder_name, fielder_id, and wicket_type being invalid in these observations. Thus, these missing data are likely classified as **missing at random(MAR)**. Given the complexity of imputing these values, they are unlikely to be included in our further analysis.

Note that, among the 28 columns, there were few attributes generated based on existing columns, and they are "over_ball", "inning", "runs_cumulative", and "powerplay". Although they are all integers, it is important to note that, only "overball" and "runs_cumulative" are quantitative, whereas "inning" and "powerplay" are categorical. Detailed calculations of these attributes are in the `cricsheet-json-parsing.ipynb` file. 

** Distribution of Variables**

![Distribution of Variables](../images/chart1.png){#fig-var_dist}

![Distribution of Variables](../images/chart2.png){#fig-var_dist_2}

**Caption:**  @fig-var_dist and @fig-var_dist_2 bar chart illustrates the distribution of all the variables. Note that the analysis excludes all columns related to `ID` as they are primarily for identification purposes and are unlikely to show significant patterns. It is observed that none of the quantitative attributes follow a normal distribution; most are right-skewed distributions. An interesting finding is noted in the attributes `over` in @fig-var_dist, where the frequency **steadily** decreases with increasing values, suggesting further analysis. Concerning ordinal attributes, we have a significantly higher amount of data from seasons that happened in recent years than in the past. 

In terms of `team` in @fig-var_dist_2, there appear to be more teams from "New Zealand," "South Africa," "India," "West India," "Pakistan," "Bangladesh," "Sri Lanka"  "Australia," and "England" than teams from other countries/regions. 

The attribute `inning` in @fig-var_dist_2 doesn't seem to have a significant difference across the two conditions, however, for `powerplay`, the number of occurrences in condition "0" is approximately twice as high as in condition "1."


In [ ]:
#| label: tbl-non_striker
#| tbl-cap: Batter Summary Count
train_data = pd.read_csv("../data/data_for_quarto/train_data.csv")
pd.concat([train_data["non_striker"].value_counts().reset_index().head(5), 
           train_data["non_striker"].value_counts().reset_index().tail(5)])

In [ ]:
#| label: tbl-batter
#| tbl-cap: Batter Summary Count
pd.concat([train_data["batter"].value_counts().reset_index().head(5), 
           train_data["batter"].value_counts().reset_index().tail(5)])

In [ ]:
#| label: tbl-bowler
#| tbl-cap: Bowler Summary Count
pd.concat([train_data["bowler"].value_counts().reset_index().head(5), 
           train_data["bowler"].value_counts().reset_index().tail(5)])

Lastly, regarding nominal attributes, we are seeing an uneven distribution appearance of players(`batter`, `bowler`, and `non-striker`) in this dataset, by using non-striker as an example, some players such as V Kohli appear around 2100 times in this dataset according to @tbl-non_striker, where other players such as F Bangur only appeared once.  Due to such a large number of batters(@tbl-batter) and bowlers(@tbl-bowler) in the dataset, we will not be using it in our analysis. To use these attributes, we will also need to create summary statistics for each batters and bowlers, as explained by @dinsdale2023explaining which is out of the scope of our project.



In general, **none of the attributes in our dataset form a normal distribution**, with samples unevenly distributed across categorical attributes. It is important to remember this when training, testing, and interpreting the model, and to appropriately acknowledge this as a potential limitation.  

### Multivariate Analysis - Correlation Analysis






![Correlation Matrix](../images/chart3.png){#fig-Correlation_Matrix}

Among the quantitative attributes in @fig-Correlation_Matrix, 5 pairs of relationship are above the threshold of weak correlation(**|r| > 0.3**), and they are: 

1. **batter_total & run_total(r = 0.97)**
2. **over & runs_cumulative(r = 0.87)**
3. **wides & runs_extras(r = 0.76)**
5. **legbyes & runs_extras(r = 0.46)**
8. **byes & runs_extra(r = 0.38)**

Note that, using Pearson's r might not be able to capture non-linear relationships that potentially exist among these relationships.


In [ ]:
#| label: tbl-wicket_count_powerplay
#| tbl-cap: Wicket Count based off of Powerplay
pd.DataFrame(train_data.groupby('powerplay')['wicket'].value_counts())

We can see that there is a much higher proportion of wickets during the powerplay (approximately 31% are wickets compared to only around 6% in non-powerplay conditions) in @tbl-wicket_count_powerplay. This could be attributed to players taking more risks since there are more fielding restrictions in the powerplay [@bhattacharjee2016impact]. While most of the game is in the powerplay, and there is a class imbalance, this suggests that the powerplay does affect the chances of a wicket.

![Wicket count across overs](../images/chart4.png){#fig-over_count_wicket}



Similarly, the distribution from @fig-over_count_wicket above shows us that towards the end of the game, there are significantly less wickets than in the first few overs, which suggests that the variable `Over` could be used for our model as well.


![Wicket count across innings](../images/chart5.png){#fig-inning_count_wicket}





The above plot @fig-inning_count_wicket indicates that there are significantly more wickets in the first innings than the second. This could be attributed to the fact that in the second innings, batters are more likely to come in with a plan as they have a target in mind to chase and they are less likely to take risks. It is also possible that the number of wickets in the second innings is lower since the teams may not play the full second innings if they reach the target before that. So, this is a variable we will take into consideration when building our model.


![Wicket Probability Density by Cumulative runs scored in current innings](../images/chart6.png){#fig-runs_cumulative}

From the @fig-runs_cumulative, there does not seem to be too much of a relationship between the runs scored so far and whether there is a wicket or not, except when the runs are greater than 100. At this point, it is more likely to be a wicket.


## Model Building

Next, we will use the relationships we found above to build our model. The variables we will be looking at are `inning`, `over`, `powerplay`, `over_ball` and `runs_cumulative`. We will perform preprocessing on our datasets using one-hot encoding on our categorical variables. While there is only one numerical variable, we will still create a scaler in case we look to add other variables later in our analysis. 

We are going to create a Logistic Regression Model for this analysis. We decided against K-NN classification as there is class imbalance and even if we balance our classes by oversampling, our accuracy will decrease using a nearest neighbours methodology. We will be using a pipeline to fit our data.

### Hyperparameter Optimization

The hyperparameter we are looking at in Logistic Regression is $C$. We will be testing for 5 different values of $C$ and looking at which gives us ideal cross-validation and training scores.


In [ ]:
#| label: tbl-hyperparameter
#| tbl-cap: Cross Validation and Training Accuracy after Hyperparameter Optimization
Hyperparameter_table = pd.read_csv("../data/data_for_quarto/Hyperparameter.csv")
Hyperparameter_table

We are choosing a $C$ of $\sqrt{10}$ as it overfits the least according to @tbl-hyperparameter .

![Distribution of Variables](../images/chart7.png){#fig-Confusion_Matrix}

# Discussion


## Summary of Results

1. Explorational Data Analysis: The exploration revealed several key insights that influence model building and interpretation. For example, the analysis shows the vast difference in player occurrences.
2. Model Building  and Selection: Based on the EDA findings, five features: Inning, over, powerplay, over_ball, and runs_cumulative were chosen based on the statistical relationships.One-hot encoding was applied for categorical variables to avoid introducing bias during model training.Finally, **Logistic Regression** was made due to the binary nature of the target variable (wicket or no wicket).Considering the potential drawbacks of class imbalance, the K-Nearest Neighbors (KNN) is not recommended.
3. Hyperparameter Optimization: Tuning the regularization parameter (C) of the Logistic Regression model to find the optimal balance between underfitting and overfitting. Testing various C values through cross-validation ensures the model generalizes well to unseen data.

## Unexpected Findings


- **Non-normality:** The extent of non-normality data across various features is unexpected. This highlights the importance of data exploration before model building.
- **Seasonal effect:** The temporal bias towards recent seasons might not be anticipated, requiring adjustments to ensure generalizability.


## Impact of Findings

These findings have several potential impacts:

- **Model development:** Understanding the distribution patterns and biases is crucial for choosing appropriate modeling techniques, interpreting outcomes, and guiding future data collection efforts.
- **Generalizability:** Addressing non-normality and temporal bias is essential for creating models that perform well on data beyond the training set.
- **Player analysis:** Exploring model results can unlock valuable insights into individual player performance and interaction with other factors influencing wicket probability.
- **Cricket analytics:** The model's insights can inform cricket analysts and fans about factors that influence wicket probabilities, leading to better predictions and understanding of the game.

## Future Questions and Work

These findings lead to several compelling questions for future research:

- **Addressing non-normality:** How can we effectively transform the data (e.g., log transformation)or utilize specific algorithms to improve model performance in the presence of *non-normal distributions*? 

- **Mitigating temporal bias:** What data augmentation techniques or alternative approaches can be used to ensure the model generalizes well across different time periods? Mitigating this bias are necessary for broader applicability.


And the future work includes:

- **Feature engineering:** Explore creating additional features or transformations of existing ones to capture more aspects of the game and potentially improve model performance.

- **Extra information:** @marshall2024talk also discuss how pitch conditions could affect the possibility of a wicket. Having this data would help improve our model significantly.

- **Model performance evaluation:** Evaluate model perform on test data using appropriate classification performance metrics (e.g., F1-score, ROC AUC) and compare it with other potential models to identify the best approach for the task.


By addressing these questions, future research can build upon the current findings and develop even more robust and insightful models for predicting wicket probabilities in T20 cricket.